In [1]:
!pip install pymysql
!pip install requests

In [2]:
import json
import pymysql as pms
import requests
import base64
import logging
import zipfile as zf
import os

In [13]:
# DB에 연결
host = "ict.cor8kkyfcogd.ap-northeast-2.rds.amazonaws.com"
port = 3306
username = "admin"
database = "ict"
password = "48834883"

try:
    conn = pms.connect(host=host,user=username,password=password,db=database, 
                       port=port, use_unicode=True, charset="utf8")
    cursor=conn.cursor()
except:
    logging.error("연결 실패")
    

In [5]:
# 현재 디렉토리 파일 목록 읽어오기
path = "./"
# ipynb 체크포인트랑 이 노트북은 제외
# 124844.json부터 85795.json까지
filelist = os.listdir(path)[1:-1]
print(filelist[0],filelist[-1])

124844.json 85795.json


In [5]:
with open (filelist[0]) as file:
    json_d = json.load(file)
    print(json_d['precedent']['casename'])

손해배상(기)등



In [15]:
# 파일명으로부터 json파일 읽어와서 sql문으로 DB에 저장 - Precedent table
for i in filelist:
    with open(i) as json_file:
        json_data = json.load(json_file)
        query = "INSERT INTO Precedent VALUES('{0}', '{1}')".format(int(json_data['precedent']['id']), json_data['precedent']['casename'])
        cursor.execute(query)
        conn.commit()

In [16]:
# 파일명으로부터 json파일 읽어와서 sql문으로 DB에 저장 - Content table
for i in filelist:
    with open(i) as json_file:
        json_data = json.load(json_file)
        idx = -1
        case_id = int(json_data['content']['id'])
        for weight in json_data['content']['key_weight']:
            idx +=1
            if weight == 0:
                continue
                
            query = "INSERT INTO Content (Precedent_ID,keyIndex,keyWeight) VALUES('{0}', '{1}', '{2}')".format(case_id, idx, weight)
            cursor.execute(query)
        
        conn.commit()

In [17]:
# 파일명으로부터 json파일 읽어와서 sql문으로 DB에 저장 - Summary table
for i in filelist:
    with open(i) as json_file:
        json_data = json.load(json_file)
        idx = -1
        case_id = int(json_data['summary']['id'])
        for weight in json_data['summary']['key_weight']:
            idx +=1
            if weight == 0:
                continue
                
            query = "INSERT INTO Summary (Precedent_ID,keyIndex,keyWeight) VALUES('{0}', '{1}', '{2}')".format(case_id, idx, weight)
            cursor.execute(query)
        
        conn.commit()

In [18]:
# 파일명으로부터 json파일 읽어와서 sql문으로 DB에 저장 - Judgement table
for i in filelist:
    with open(i) as json_file:
        json_data = json.load(json_file)
        idx = -1
        case_id = int(json_data['judgement']['id'])
        for weight in json_data['judgement']['key_weight']:
            idx +=1
            if weight == 0:
                continue
                
            query = "INSERT INTO Judgement (Precedent_ID,keyIndex,keyWeight) VALUES('{0}', '{1}', '{2}')".format(case_id, idx, weight)
            cursor.execute(query)
        
        conn.commit()

In [14]:
# 싹다 지웁니다.
query = "delete from Content where Precedent_ID>=0"
cursor.execute(query)
print("Suceed")
query = "DELETE FROM Precedent where ID>=0"
cursor.execute(query)
print("Suceed")
conn.commit()

Suceed
Suceed
